In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense
import pickle
import numpy as np
from keras.utils import Sequence
import mmap 

In [2]:
tfidf = Input(shape=(1000,))
x1 = Dense(128, activation='relu')(tfidf)
x1 = Dense(64, activation='relu')(x1)
outputs = Dense(1, activation='sigmoid')(x1)
model = Model(inputs=tfidf, outputs=outputs)

In [3]:
mch = callbacks.ModelCheckpoint('../models/fianl-model-tfidf.h5' , monitor='accuracy' , mode ='max' , save_best_only=True)


In [4]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
     


In [5]:
class DataGenerator(Sequence):
    def __init__(self,dataset_size, batch_size, x_file, y_file):
        self.dataset_size = dataset_size
        self.batch_size = batch_size
        self.x_file = x_file
        self.y_file = y_file

        with open(x_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_train = np.load(mm , allow_pickle=True)

        with open(y_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_test = np.load(mm , allow_pickle=True)

    def __len__(self):
        return int(np.ceil(self.dataset_size / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_X = self.arr_train[idx * self.batch_size:(idx + 1) * self.batch_size].toarray()
        batch_y = self.arr_test[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_X, batch_y

batch_size = 128
dataset_size_train = 3600000
dataset_size_test = 400000

train_generator = DataGenerator(dataset_size_train , batch_size, '/content/drive/MyDrive/persian-sentiment-analysis/X_train_tfidf1.pickle', '/content/drive/MyDrive/persian-sentiment-analysis/train_labels.pickle')
test_generator = DataGenerator(dataset_size_test , batch_size, '/content/drive/MyDrive/persian-sentiment-analysis/X_test_tfidf.pickle', '/content/drive/MyDrive/persian-sentiment-analysis/test_labels.pickle')


     

In [6]:
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10 , validation_data=test_generator, validation_steps=len(test_generator),callbacks=[mch])


Epoch 1/10
28125/28125 [==============================] - 138s 5ms/step - loss: 0.2624 - accuracy: 0.8892 - val_loss: 0.2461 - val_accuracy: 0.8974
Epoch 2/10
28125/28125 [==============================] - 130s 5ms/step - loss: 0.2382 - accuracy: 0.9010 - val_loss: 0.2396 - val_accuracy: 0.9002
Epoch 3/10
28125/28125 [==============================] - 132s 5ms/step - loss: 0.2290 - accuracy: 0.9055 - val_loss: 0.2395 - val_accuracy: 0.9007
Epoch 4/10
28125/28125 [==============================] - 130s 5ms/step - loss: 0.2232 - accuracy: 0.9081 - val_loss: 0.2387 - val_accuracy: 0.9008
Epoch 5/10
28125/28125 [==============================] - 130s 5ms/step - loss: 0.2192 - accuracy: 0.9102 - val_loss: 0.2393 - val_accuracy: 0.9006
Epoch 6/10
28125/28125 [==============================] - 130s 5ms/step - loss: 0.2160 - accuracy: 0.9115 - val_loss: 0.2398 - val_accuracy: 0.9002
Epoch 7/10
28125/28125 [==============================] - 131s 5ms/step - loss: 0.2136 - accuracy: 0.9127 - val_

In [ ]:
b